In [1]:
import json, csv
import pandas as pd

In [2]:
with open("output/enhanced_dataset.json", 'r') as fp:
    dataset = json.load(fp)

In [3]:
admin = pd.read_csv('resources/admin_areas.csv',index_col='uniq_id')

In [4]:
cyrillic_translit={u'\u0410': 'A', u'\u0430': 'a',u'\u0411': 'B', u'\u0431': 'b',u'\u0412': 'V', u'\u0432': 'v',u'\u0413': 'G', u'\u0433': 'g',u'\u0414': 'D', u'\u0434': 'd',u'\u0415': 'E', u'\u0435': 'e',u'\u0416': 'Zh', u'\u0436': 'zh',u'\u0417': 'Z', u'\u0437': 'z',u'\u0418': 'I', u'\u0438': 'i',u'\u0419': 'I', u'\u0439': 'i',u'\u041a': 'K', u'\u043a': 'k',u'\u041b': 'L', u'\u043b': 'l',u'\u041c': 'M', u'\u043c': 'm',u'\u041d': 'N', u'\u043d': 'n',u'\u041e': 'O', u'\u043e': 'o',u'\u041f': 'P', u'\u043f': 'p',u'\u0420': 'R', u'\u0440': 'r',u'\u0421': 'S', u'\u0441': 's',u'\u0422': 'T', u'\u0442': 't',u'\u0423': 'U', u'\u0443': 'u',u'\u0424': 'F', u'\u0444': 'f',u'\u0425': 'Kh', u'\u0445': 'kh',u'\u0426': 'Ts', u'\u0446': 'ts',u'\u0427': 'Ch', u'\u0447': 'ch',u'\u0428': 'Sh', u'\u0448': 'sh',u'\u0429': 'Shch', u'\u0449': 'shch',u'\u042a': '"', u'\u044a': '"',u'\u042b': 'Y', u'\u044b': 'y',u'\u042c': "'", u'\u044c': "'",u'\u042d': 'E', u'\u044d': 'e',u'\u042e': 'Iu', u'\u044e': 'iu',u'\u042f': 'Ia', u'\u044f': 'ia',u'\u0462': 'E', u'\u0463': 'e'}

def transliterate(word, translit_table):
    """
    Transliterates 'word' based on the key/value pairs in 'translit_table'
    """
    if type(word)!=str:
        return None
    else:
        converted_word = ''
        for char in word:
            transchar = ''
            if char in translit_table:
                transchar = translit_table[char]
            else:
                transchar = char
            converted_word += transchar
        return converted_word

In [5]:
dataset["528"]

{'SOURCE': 'http://babel.hathitrust.org/cgi/pt?id=uc1.b4583201;view=1up;seq=87',
 'admin1_new_orth': 'Екатеринославского Наместничества',
 'admin1_old_orth': 'Екатеринославскаго Намѣстничества',
 'admin1_partofID': 8.0,
 'admin1_std_name': 'Екатеринославское Наместничество',
 'admin1_stem': 'екатеринославск',
 'admin2_new_orth': None,
 'admin2_old_orth': None,
 'admin2_partofID': None,
 'admin2_stem': 'not_in_text',
 'admin_hierarchy': 'екатеринославск|not_in_text',
 'alt_name_new_orth': None,
 'alt_name_old_orth': None,
 'desc_machine_translated': 'PETROVSKAYA fortress Yekaterinoslav region. In the case thereof, in the year six fairs.',
 'description_new_orth': 'ПЕТРОВСКАЯ , крепость Екатеринославского Наместничества. В оной бывает в году шесть ярмарок.',
 'description_old_orth': 'ПЕТРОВСКАЯ , крѣпость Екатеринославскаго Намѣстничества. Въ оной бываетъ въ году шесть ярмарокъ.',
 'featuretype_new_orth': 'крепость',
 'featuretype_new_orth_lc': "krepost'",
 'featuretype_old_orth': 'крѣпо

In [6]:
sourceLink = "https://www.zotero.org/jaguillette/items/collectionKey/3KQ5I42Q/itemKey/5AI864SV"
startDate = 1785
endDate = 1796
confidence = "confident"

In [7]:
def add_admin_to_places(adminID,adminType,writer=place_csv_writer,admin_df=admin):
    name = admin_df.loc[int(adminID)]['lc_translit_new']
    lat = admin_df.loc[int(adminID)]['y_coord']
    lon = admin_df.loc[int(adminID)]['x_coord']
    adminID = int(adminID)+805
    place_csv_writer.writerow([adminID,name,lat,lon,'seat',adminType,0])
    return

NameError: name 'place_csv_writer' is not defined

In [14]:
with open("graph/(places).csv","w+") as place_fp, open("graph/[partof].csv","w+") as partof_fp:
    # Setup place csv writer and headers
    place_csv_writer = csv.writer(place_fp)
    place_csv_writer.writerow(['ID','title','reprLat','reprLong','reprType','geonamesType','provisional'])
    # Setup partof csv writer and headers
    partof_csv_writer = csv.writer(partof_fp)
    partof_csv_writer.writerow(['ID','child','parent','sourceLink','startDate','endDate','confidence'])
    def add_admin_to_places(adminID,adminType,writer=place_csv_writer,admin_df=admin):
        name = admin_df.loc[int(adminID)]['lc_translit_new']
        lat = admin_df.loc[int(adminID)]['y_coord']
        lon = admin_df.loc[int(adminID)]['x_coord']
        adminID = int(adminID)+805
        place_csv_writer.writerow([adminID,name,lat,lon,'seat',adminType,0])
        return
    # keep track of admin places entered
    partof_places_entered = []
    for k,v in dataset.items():
        if 'admin2_partofID' in v and v['admin2_partofID']:
            admin2_id = int(v['admin2_partofID']) + 805
            # Check if admin2 recorded, add if not
            if v['admin2_partofID'] not in partof_places_entered:
                try:
                    add_admin_to_places(v['admin2_partofID'],'ADM2')
                except KeyError:
                    print("KeyError! ({})".format(v['admin2_partofID']))
                    print(v)
                    print()
                partof_places_entered.append(v['admin2_partofID'])
            # Add partof relationship  place -> admin2
            partof_csv_writer.writerow([None,k,admin2_id,sourceLink,startDate,endDate,confidence])
            if 'admin1_partofID' in v and v['admin1_partofID']:
                admin1_id = int(v['admin1_partofID']) + 805
                # Check if admin1 recorded, add if not
                if v['admin1_partofID'] not in partof_places_entered:
                    add_admin_to_places(v['admin1_partofID'],'ADM1')
                    partof_places_entered.append(v['admin1_partofID'])
                # Add partof relationship place -> admin1 and admin2 -> admin1
                partof_csv_writer.writerow([None,k,admin1_id,sourceLink,startDate,endDate,confidence])
                partof_csv_writer.writerow([None,admin2_id,admin1_id,sourceLink,startDate,endDate,confidence])
            else:
                print("The following has an admin2 ID, but no admin1 ID:")
                print(v['description_new_orth']) # make sure that's right, since it's weird
                print(v['desc_machine_translated'])
                print()
        else:
            if 'admin1_partofID' in v and v['admin1_partofID']:
                admin1_id = int(v['admin1_partofID']) + 805
                # Check if admin1 recorded, add if not
                if v['admin1_partofID'] not in partof_places_entered:
                    add_admin_to_places(v['admin1_partofID'],'ADM1')
                    partof_places_entered.append(v['admin1_partofID'])
                # Add partof relationship place -> admin1
                partof_csv_writer.writerow([None,k,admin1_id,sourceLink,startDate,endDate,confidence])
            else:
                print("The following has no admin partof IDs:")
                print(v['description_new_orth']) # make sure that's right, since it's weird
                print(v['desc_machine_translated'])
                print()
        # Add place to Places
        if 'geo' in v:
            lon = v['geo']['x_coord']
            lat = v['geo']['y_coord']
            reprType = 'GeoNames'
        else:
            lon = None
            lat = None
            reprType = None
        place_csv_writer.writerow([k,v['name_new_orth_lc'],lat,lon,reprType,'PPL',0])

The following has no admin partof IDs:
МИТЯКИНСКАЯ , станица Донских Козаков. В ней всякой год в десятую пятницу по Пасхе бывает ярмарка, на которую пригоняют скот , также привозят вялую рыбу , холст, шелковые материи, сукна, деготь и горячие напитки.
MITYAKINSKAYA, village Don Cossacks. In it every year on the tenth Friday after Easter is fair, where cattle; also bring sluggish fish, canvas, silk fabrics, cloth, tar and hot drinks.

The following has no admin partof IDs:
ОЛИШЕВКА , местечко в округе города Киева. Здесь бывает в году две ярмарки ; на оных скупают приезжие купцы кожи , овчины , шубы , соль и прочее.
OLISHEVKA, town in the district of Kiev. Here is the year two fairs; buy add-on visiting merchants leather, sheepskin, fur coats, salt and so on.

The following has no admin partof IDs:
ЧЕРЕНЧИЦКОЙ, погост в уезде города Старой Рузы. Здесь бывает одна годовая ярмарка в день Ильинской пятницы, и купцы приезжают из Старой Рузы с мелочными товарами.
CHERENCHITSKOY, churchyard i

In [9]:
# Names
with open('graph/(names).csv','w+') as fpn, open('graph/[called].csv','w+') as fpc:
    n_writer = csv.writer(fpn)
    c_writer = csv.writer(fpc)
    n_writer.writerow(['ID','attestedName','romanized_name','nameLanguage'])
    c_writer.writerow(['ID','place','name','sourceLink','startDate','endDate','confidence'])
    # Every entry gets its name attested
    nameID = 1
    calledID = 1
    for k,v in dataset.items():
        n_writer.writerow([nameID,v['name_old_orth'],v['name_old_orth_lc'],'Russian (Old Orthography)'])
        c_writer.writerow([calledID,k,nameID,sourceLink,startDate,endDate,confidence])
        calledID += 1
        nameID += 1
        if v['alt_name_old_orth']:
            romanizedName = transliterate(v['alt_name_old_orth'],cyrillic_translit)
            n_writer.writerow([nameID,v['alt_name_old_orth'],romanizedName,'Russian (Old Orthography)'])
            c_writer.writerow([calledID,k,nameID,sourceLink,startDate,endDate,confidence])
            nameID += 1
            calledID += 1

In [15]:
import os

In [20]:
filelist = os.listdir('geonames_jsons')

['105_Веневской_Николаевской_f0.3.json',
 '118_Волокопинежская_f0.6.json',
 '120_Волчья.json',
 '121_Волынцы.json',
 '122_Вольная_f0.5.json',
 '131_Высокое.json',
 '132_Высокое.json',
 '134_Вычуга_f0.5.json',
 '135_Вышгород_f0.8.json',
 '136_Вышгород.json',
 '138_Ветка.json',
 '13_Алексеевская_f0.8.json',
 '141_Вязовна_f0.8.json',
 '143_Гаврилова.json',
 '144_Гагарино.json',
 '144_Гагарино_f.9.geojson',
 '146_Галяни_f0.8.json',
 '147_Павшино.json',
 '149_Геннадиев_f0.4.json',
 '14_Алешня.json',
 '150_Германова_f0.6.json',
 '152_Глемязов_f0.7.json',
 '156_Глушицкий.json',
 '158_Гнильской_f0.5.json',
 '162_Гомля_f0.6.json',
 '166_Городище.json',
 '167_Городище.json',
 '168_Городня.json',
 '170_Городок.json',
 '173_Гребенишки_f0.7.json',
 '175_Гремячь.json',
 '177_Грунь.json',
 '17_Антоновка.json',
 '180_Густиново_f0.6.json',
 '182_Даймик_f0.4.json',
 '182_Даймино_f0.7.json',
 '187_Дергачи.json',
 '188_Деркул.json',
 '18_Апичерская_f0.3.json',
 '192_Дмитровка.json',
 '193_Дмитровска

In [19]:
not_found_list = ["4","5","6","7","14","17","18","19","21","23","24","29","30","98","33","34","36","37","85","86","88","90","92","93","41","43","46","49","50","52","56","58","59","60","61","74","76","77","78","80","105","138","118","120","122","131","132","134","136","141","144","147","149","150","152","158","162","167","166","168","170","173","177","180","182","209","187","188","192","193","195","199","200","201","202","205","206","211","212","215","222","221","223","232","237","242","243","246","245","249","247","251","252","253","254","257","265","266","262","264","267","269","272","277","279","283","285","298","299","287","292","295","296","281","301","304","307","308","312","316","318","322","323","324","330","338","340","339","342","344","347","349","351","352","353","357","358","362","364","366","367","368","372","374","377","395","380","382","383","384","390","391","401","403","409","411","412","413","418","420","423","424","426","434","435","436","441","449","452","453","457","459","460","465","469","470","474","473","478","483","485","486","487","489","492","496","501","499","500","502","508","510","513","514","515","517","519","525","529","530","531","535","536","539","540","542","543","546","548","553","555","568","570","574","577","580","585","587","592","593","599","600","602","607","610","611","614","617","624","625","630","631","633","634","636","638","641","644","645","648","651","654","662","665","666","675","677","679","683","684","685","686","692","690","700","702","704","706","708","709","712","713","714","721","725","726","727","730","731","733","735","736","740","741","746","747","749","750","751","753","754","759","765","763","768","776","777","780","781","783","784","786","792","795","802","804"]

In [33]:
output = {}
for f in filelist:
    file_id = f.split('_')[0]
    if file_id in not_found_list:
        if file_id not in output:
            output[int(file_id)] = []
        output[int(file_id)].append("https://github.com/FungDavis/fungHGR/blob/master/geonames_jsons/{}".format(f))

In [23]:
import collections

In [34]:
od = collections.OrderedDict(sorted(output.items()))
od

OrderedDict([(4,
              ['https://github.com/FungDavis/fungHGR/blob/master/geonames_jsons/4_Авженская_f0.7.json']),
             (5,
              ['https://github.com/FungDavis/fungHGR/blob/master/geonames_jsons/5_Аврамиев_f0.4.json']),
             (6,
              ['https://github.com/FungDavis/fungHGR/blob/master/geonames_jsons/6_Адоватово_f0.5.json']),
             (7,
              ['https://github.com/FungDavis/fungHGR/blob/master/geonames_jsons/7_Аксель.json']),
             (14,
              ['https://github.com/FungDavis/fungHGR/blob/master/geonames_jsons/14_Алешня.json']),
             (17,
              ['https://github.com/FungDavis/fungHGR/blob/master/geonames_jsons/17_Антоновка.json']),
             (18,
              ['https://github.com/FungDavis/fungHGR/blob/master/geonames_jsons/18_Апичерская_f0.3.json']),
             (19,
              ['https://github.com/FungDavis/fungHGR/blob/master/geonames_jsons/19_Аргамачья_f0.6.json']),
             (21,
           

In [35]:
for k,v in od.items():
    print(', '.join(v))

https://github.com/FungDavis/fungHGR/blob/master/geonames_jsons/4_Авженская_f0.7.json
https://github.com/FungDavis/fungHGR/blob/master/geonames_jsons/5_Аврамиев_f0.4.json
https://github.com/FungDavis/fungHGR/blob/master/geonames_jsons/6_Адоватово_f0.5.json
https://github.com/FungDavis/fungHGR/blob/master/geonames_jsons/7_Аксель.json
https://github.com/FungDavis/fungHGR/blob/master/geonames_jsons/14_Алешня.json
https://github.com/FungDavis/fungHGR/blob/master/geonames_jsons/17_Антоновка.json
https://github.com/FungDavis/fungHGR/blob/master/geonames_jsons/18_Апичерская_f0.3.json
https://github.com/FungDavis/fungHGR/blob/master/geonames_jsons/19_Аргамачья_f0.6.json
https://github.com/FungDavis/fungHGR/blob/master/geonames_jsons/21_Артемоновка_f0.7.json
https://github.com/FungDavis/fungHGR/blob/master/geonames_jsons/23_Архангельское.json
https://github.com/FungDavis/fungHGR/blob/master/geonames_jsons/24_Асаково_f0.7.json
https://github.com/FungDavis/fungHGR/blob/master/geonames_jsons/29_Ба